## Imports

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D,Activation,Input
from keras.models import Model
from keras import regularizers
from keras.regularizers import l1, l2
from keras.callbacks import Callback

import csv
import json

## Generate Model

In [9]:
def add_dense_layer(model, layer):
    type_of_activation = str(int(layer[2]))
    act = params['type_of_activation_in_dense_lyrs'].get(type_of_activation)
    # print("params['type_of_activation_in_dense_lyrs']:", params['type_of_activation_in_dense_lyrs'], "layer[2]=",layer[2], ": act=",act)
    if layer[3] == 0:
        model.add(Dense(units=layer[1], 
                    activation=act))
    elif layer[3] == 1:
        reg = regularizers.l1(layer[4])
        model.add(Dense(units=layer[1], 
                    activation=act,
                    kernel_regularizer=reg))
    elif layer[3] == 2:
        reg = regularizers.l2(layer[4])
        model.add(Dense(units=layer[1], 
                    activation=act,
                    kernel_regularizer=reg))   

In [10]:
def add_compile_layer(model,layer):
    optimizer = None
    if layer[1] == 0:
        optimizer = Adam(learning_rate=layer[2])
    elif layer[1] == 1:
        optimizer = SGD(learning_rate=layer[2])
    elif layer[1] == 2:
        optimizer = RMSprop(learning_rate=layer[2])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
def add_conv2d_layer(model, layer):
    type_of_activation = str(int(layer[3]))
    activation = params['type_of_activation_in_conv_lyrs'].get(type_of_activation)
    # print(activation)
    if layer[6] > 0:
        if layer[4] == 0:
            model.add(Conv2D(filters=layer[1], 
                     kernel_size=int(layer[2]),
                     padding='same',
                     activation=activation, 
                     input_shape=(48, 48,1)))
        elif layer[4] == 1:
            reg = regularizers.l1(layer[5])
            model.add(Conv2D(filters=layer[1],
                    kernel_size=int(layer[2]),
                    padding='same',
                    activation=activation, 
                    kernel_regularizer=reg,
                    input_shape=(48, 48,1)))
        elif layer[4] == 2:
            reg = regularizers.l2(layer[5])
            model.add(Conv2D(filters=layer[1],
                    kernel_size=int(layer[2]), 
                    padding='same',
                    activation=activation, 
                    kernel_regularizer=reg,
                    input_shape=(48, 48,1)))
    else:
        if layer[4] == 0:
            model.add(Conv2D(filters=layer[1], 
                     kernel_size=int(layer[2]),
                     padding='same',
                     activation=activation))
        elif layer[4] == 1:
            reg = regularizers.l1(layer[5])
            model.add(Conv2D(filters=layer[1],
                    kernel_size=int(layer[2]), 
                    padding='same',
                    activation=activation, 
                    kernel_regularizer=reg))
        elif layer[4] == 2:
            reg = regularizers.l2(layer[5])
            model.add(Conv2D(filters=layer[1],
                    kernel_size=int(layer[2]), 
                    padding='same',
                    activation=activation, 
                    kernel_regularizer=reg))


In [12]:

def add_batch_norm_layer(model, layer):
    model.add(BatchNormalization())

def add_maxpool2d_layer(model, layer):
        model.add(MaxPool2D(pool_size=(2, 2)))

def add_dropout_layer(model, layer):
    model.add(Dropout(rate=layer[1]))

def add_flatten_layer(model, layer):
    model.add(Flatten())
    
def add_fit_layer(model, layer):
    return


In [13]:
# Then, you can create a dictionary to map layer types to functions
layer_functions = {
    0: add_conv2d_layer,
    1: add_batch_norm_layer,
    2: add_maxpool2d_layer,
    3: add_dropout_layer,
    4: add_dense_layer,
    5: add_flatten_layer,
    6: add_compile_layer,
    7: add_fit_layer
}

In [1]:

def generate(genome,verbose):
    model = Sequential()
    for layer in genome:
        layer_functions[layer[0]](model, layer)
    
    if verbose:
        model.summary()
        # config = model.get_config()
        # i = 0
        # for c in config['layers']:
        #     i += 1
        #     print(f"\nlayer {i}: {c}")
    num_trainable_params = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    return model, num_trainable_params

In [22]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)